# 무신사 스크래이핑
* [x] 상품번호
* [x] 브랜드
* [x] 제품명
* [x] 가격
* [x] 제품 만족도
* [x] 썸네일 사진 - 001001-상품번호-thumb.jpg
* [x] 제품 상세 사진 -  001001-상품번호-detail-{n}.jpg

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve # urlretrieve : 이미지 다운

url = 'https://store.musinsa.com/app/items/lists/001001'

In [8]:
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

In [9]:
li_box = soup.find('ul', id='searchList').find_all('li', class_='li_box')
len(li_box)

90

In [11]:
brand   = li_box[0].find('p', class_='item_title').text
product = li_box[0].find('p', class_='list_info').text
price   = li_box[0].find('p', class_='price').text

In [12]:
brand, product, price

('그레이버',
 '\n\n\t\t\t\t[UNISEX]도트보드 자수 화이트클립 반팔티_2color\n',
 '\n\t\t\t\t\t\t\t\t42,000원\n\t\t\t')

In [15]:
# 상품번호 가져오기
product_num = li_box[0].find('div', class_='list_img').find('a')['href'].split('/')[-2]
product_num

'1517122'

# 썸네일 저장하기
 - path : ./musinsa/thumb

In [16]:
thumb_path = './musinsa/thumb/{}'

In [19]:
# 썸네일 url 가지고 오기
thumb_url = li_box[0].find('div', class_='list_img').find('a').find('img')['data-original']
thumb_url

'//image.msscdn.net/images/goods_img/20200716/1517122/1517122_1_125.jpg'

In [23]:
thumb_img_name = '001001-{}-thumb.jpg'.format(product_num)
urlretrieve('https:'+thumb_url, thumb_path.format(thumb_img_name))

('./musinsa/thumb/001001-1517122-thumb.jpg',
 <http.client.HTTPMessage at 0x159cd85e088>)

# 상세 페이지 url

In [24]:
detail_url = li_box[0].find('div', class_='list_img').find('a')['href']
detail_url

'/app/product/detail/1517122/0'

In [25]:
from urllib.parse import urljoin

base_url = 'https://store.musinsa.com/'

full_url = urljoin(base_url, detail_url)
full_url

'https://store.musinsa.com/app/product/detail/1517122/0'

In [27]:
detail_page = urlopen(full_url)
detail_soup = BeautifulSoup(detail_page, 'html.parser')

# 후기 가지고 오기

In [29]:
review_msg = detail_soup.find('div', class_='estimate-stats').text.strip()
review_msg

'등록된 후기가 없습니다.'

# 상세 이미지 가져오기
 - 규칙 확인 결과 : thum_[li의 인덱스]로 계속 늘어가는 것을 확인

In [32]:
img_li_list = detail_soup.find('ul', class_='product_thumb').find_all('li')
len(img_li_list)

1

In [38]:
img_id_format = 'thum_{}'

In [94]:
detail_img_name = '001001-{}-detail-{}.jpg'
detail_img_path = "./musinsa/detail/{}"
for idx, li in enumerate(img_li_list):
    img = li.find('img', id=img_id_format.format(idx))
    img_url = img['src']
    img_url_split = img_url.split('/')
    img_original_name = img_url_split[-1]
    
    img_url_complete = 'https:'+'/'.join(img_url_split[:-1])+"/"+img_original_name[:-6]+"500.jpg"
    urlretrieve(img_url_complete, detail_img_path.format(detail_img_name.format(product_num, idx)))

# 1. 한 페이지의 전체 상품 크롤링

In [99]:
import pandas as pd
import tqdm 
base_url = 'https://store.musinsa.com'
list_url = '/app/items/lists/001001'

url = 'https://store.musinsa.com/app/items/lists/001001'
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

thumb_path  = './musinsa/thumb/{}'
detail_path = "./musinsa/detail/{}"

thumb_img_name  = '001001-{}-thumb.jpg'
detail_img_name = '001001-{}-detail-{}.jpg'

img_id_format = 'thum_{}'

li_box = soup.find('ul', id='searchList').find_all('li', class_='li_box')

product_num_list = []
brand_list        = []
product_name_list = []
price_list        = []
review_list       = []
thumb_list        = []
detail_list       = []



for li in tqdm.tqdm_notebook(li_box):
    brand   = li.find('p', class_='item_title').text
    product = li.find('p', class_='list_info').text
    price   = li.find('p', class_='price').text
    product_num = li.find('div', class_='list_img').find('a')['href'].split('/')[-2]
    thumbnail = thumb_img_name.format(product_num)
    
    product_num_list.append(product_num)
    brand_list.append(brand)
    product_name_list.append(product)
    price_list.append(price)
    thumb_list.append(thumbnail)
    
    # 이미지 다운로드
    thumb_url = li.find('div', class_='list_img').find('a').find('img')['data-original']
    urlretrieve('https:'+thumb_url, thumb_path.format(thumbnail))
    
    detail_url = li.find('div', class_='list_img').find('a')['href']
    
    # 디테일 페이지 크롤링 시작
    full_url = urljoin(base_url, detail_url)
    detail_page = urlopen(full_url)
    detail_soup = BeautifulSoup(detail_page, 'html.parser')
    
    # 디테일 페이지에서 후기 가져오기
    review_msg = detail_soup.find('div', class_='estimate-stats').text.strip()
    review_list.append(review_msg)
    
    # 상세 이미지 목록 가져오기
    img_li_list = detail_soup.find('ul', class_='product_thumb').find_all('li')
    
    detail_img_list = []
    for idx, li in enumerate(img_li_list):
        img = li.find('img', id=img_id_format.format(idx))
        img_url = img['src']
        img_url_split = img_url.split('/')
        img_original_name = img_url_split[-1]

        img_url_complete = 'https:'+'/'.join(img_url_split[:-1])+"/"+img_original_name[:-6]+"500.jpg"
        detail_name = detail_img_name.format(product_num, idx)
        detail_img_list.append(detail_name)
        urlretrieve(img_url_complete, detail_img_path.format(detail_name))
        
    detail_list.append('#'.join(detail_img_list))
    
df_musinsa = pd.DataFrame({
    '제품번호':product_num_list,
    '브랜드':brand_list,
    '제품명':product_name_list,
    '가격':price_list,
    '썸네일': thumb_list,
    '상세사진': detail_list
})
df_musinsa.to_csv('./data/musinsa.csv', encoding='utf-8')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


NameError: name 'df' is not defined